In [1]:
import sys

import numpy as np
from dataset import *

import argparse

import os

current_script_path = os.getcwd()
main_folder_path = os.path.dirname(current_script_path)

if main_folder_path not in sys.path:
    sys.path.append(main_folder_path)


from domains.gridworld import *
from generators.obstacle_gen import *
from tqdm import tqdm 

import numpy as np
import matplotlib.pyplot as plt
import pykonal




In [2]:
# Number of trials
num_trials =1200 # 1000 for training and 200 for testing
env_size = 28

x_values_array = np.zeros((env_size* env_size* 1, num_trials))
y_values_array = np.zeros((env_size* env_size* 1, num_trials))
velocity_matrices_array = np.zeros((num_trials, env_size, env_size))
inverse_travel_time_values_array = np.zeros((num_trials, env_size, env_size))

for trial in tqdm(range(num_trials)):
# Instantiate EikonalSolver object using Cartesian coordinates.
    solver = pykonal.EikonalSolver(coord_sys="cartesian")
    solver.velocity.min_coords = 0, 0, 0
    solver.velocity.node_intervals = 1, 1, 1
    solver.velocity.npts = env_size, env_size, 1

    # Create a velocity matrix with all values set to 1.0
    velocity_matrix = np.ones((env_size, env_size, 1))
     
    goal = [14,14]
    max_obs=20
    max_obs_size=6
    dom_size=(env_size, env_size)

    obs = obstacles([dom_size[0], dom_size[1]], goal, max_obs_size)
    n_obs = obs.add_n_rand_obs(max_obs)
    border_res = obs.add_border()
    im = obs.get_final()
    # Generate gridworld from obstacle map
    G = GridWorld(im, goal[0], goal[1])
    im_np = np.array(im).reshape(env_size, env_size, 1)

    solver.velocity.values = im_np
    src_idx = goal[1],goal[0], 0
    solver.traveltime.values[src_idx] = 100

    # Set the unknown flag for the source node to False.
    solver.unknown[src_idx] = False
    solver.trial.push(*src_idx)

    # Solve the system.
    solver.solve()

    velocity_matrices_array[trial,:,:] = im_np.reshape(env_size,env_size)
    inverse_travel_time_values_array[trial,:, :] = 1/solver.traveltime.values[:, :, 0]


100%|██████████| 1200/1200 [01:19<00:00, 15.18it/s]


In [3]:
# Save velocity_matrices_array as "mask.npy"
np.save("mask.npy", velocity_matrices_array)

# Save travel_time_values_array as "output.npy"
np.save("output.npy", inverse_travel_time_values_array)

In [4]:
def calculate_signed_distance(velocity_matrix):
    # Find the indices of obstacles
    obstacle_indices = np.where(velocity_matrix < 1.0)

    # Check if there are no obstacles
    if obstacle_indices[0].size == 0:
        # Handle the case when there are no obstacles
        return np.zeros_like(velocity_matrix)

    # Create a meshgrid of indices
    i, j = np.meshgrid(np.arange(velocity_matrix.shape[1]), np.arange(velocity_matrix.shape[0]), indexing='ij')  # Note the change in the order of indices
    
    # Calculate distances to obstacles using vectorized operations
    distances = np.sqrt((obstacle_indices[0][:, None, None] - i[None, :, :])**2 + (obstacle_indices[1][:, None, None] - j[None, :, :])**2)

    # Find the minimum distance for each point
    signed_distance = np.min(distances, axis=0)
    return signed_distance



In [5]:

signed_distance_array = np.zeros((num_trials, env_size, env_size))

for i in tqdm(range(num_trials)):
    signed_distance_array[i,:,:] = calculate_signed_distance(velocity_matrices_array[i,:,:].reshape(env_size,env_size,1))

# Save signed_distance_array as "dist_in.npy"
np.save("dist_in.npy", signed_distance_array)

100%|██████████| 1200/1200 [00:01<00:00, 1131.84it/s]
